In [1]:
import sys
import time
import pickle
import pandas as pd
from pprint import pprint
from collections import Counter

In [2]:
def unpickle(filename):
    ''' open file '''
    with open(filename, 'r') as picklefile:
        old_data = pickle.load(picklefile)
    return old_data

In [3]:
basedir = "./pkls/"

In [4]:
kmeans = unpickle(basedir + "kmeans40.pkl")

In [5]:
with open(basedir + "zscore_df_csv.csv", 'r') as tf:
    tracks = pd.read_csv(tf, index_col=0)

In [6]:
with open(basedir + "titles_csv3.csv", 'r') as tf:
    titles = pd.read_csv(tf, index_col=0)

In [7]:
titles.head()

,track_id,title,artist_id,artist,song_id
index,,,,,
0,TRBGPHG12903CE6CC3,The Law Gonna Step On You (1931),ARFXRHR1187B98FF09,Bo Carter,SORUUEV12A58A7B9FA
1,TRBGPYK128F42796E1,Des Vôtres,ARQDTOS12086C11443,Fredericks_ Goldman_ Jones,SOEZDOH12A8AE4787A
2,TRBGPJP128E078ED20,Crazy,AR12F2S1187FB56EEF,Aerosmith,SOOOWIC12A6701C7E5
3,TRBGPXH128F428C912,Chiove,AR7LIU31187B98EF11,Rita Chiarelli,SOLSWBA12A8C141B9A
4,TRBGPSV12903CA9C25,Rude Bwoy Love (feat. Dj Fly_ Dj Traxx_ T.Will...,ARPGCHN1187B9A2831,Nicky B_ Naëlle,SOHNJQL12AB018CC5C


In [8]:
with open("mismatch_tracks.txt", 'r') as mtf:
    bad_tracks = mtf.read().split()

print bad_tracks[:5]

btset = set(bad_tracks)

['TRAAAJO12903CAAC69', 'TRAAAPK128E0786D96', 'TRAAEJQ128F92C484E', 'TRAAGPJ128F428CD1B', 'TRAAHER128F92D0354']


###When matching on trackid

In [4]:
#mbid_tracks = {}
#with open("mbid_tracks_1.csv", 'r') as mtf:
#    for line in mtf:
#        line = line.strip()
#        segs = line.split(",")
#        key = segs[0]
#        track = segs[1]
#        mbid_tracks[key] = track

In [5]:
#print len(mbid_tracks.keys())

In [38]:
#usertracks = []
#with open("lastfm_usertracks.txt", 'r') as lutf:
#    for line in lutf:
#        line = line.strip()
#        uts = []
#        for l in line.split():
#            uts.append(l)
#        usertracks.append(uts)

In [89]:
#userclusters = []
#t1 = time.time()
#for uts in usertracks:
#    ucs = []
#    for trackid in uts:
#        if trackid in tracks.track_id.values:
#            print trackid
#            track = tracks[tracks["track_id"] == trackid]
#            track = track.iloc[:,1:] #leave out track_id
#            cluster = kmeans.predict(track)
#            ucs.append(cluster[0])#only one track used in predict
#    userclusters.append(ucs) 
#print "predict time:", time.time()-t1

predict time: 32.6945610046


###For matching on songid

In [59]:
mbid_songs = {}
with open("mbid_songs.csv", 'r') as msf:
    for line in msf:
        line = line.strip()
        segs = line.split(",")
        key = segs[0]
        track = segs[1]
        mbid_songs[key] = track

In [60]:
print len(mbid_songs.keys())

1226050


In [61]:
usersongs = []
linelimit = 10000
currentline = 0
with open("mbids.txt", 'r') as mf:
    for line in mf:
        if currentline >= linelimit:
            break
        else:
            uss = []
            for mbid in line.split():
                song = mbid_songs[mbid]
                #print track
                if song != "":
                    #print track
                    uss.append(song)
            usersongs.append(uss)
        currentline += 1

In [62]:
with open("lastfm_usersongs.txt", 'w') as lusf:
    for uss in usersongs:
        #print "hello"
        for song in uss:
            lusf.write(song + " ")
        lusf.write("\n")

In [39]:
#print usertracks[0]
#print usersongs[0]

In [24]:
#known song in dataset
#usersongs.append(['SOOOWIC12A6701C7E5'])

In [63]:
userclusters = []
t1 = time.time()
known_songs = set(titles.song_id.values)
for uss in usersongs:
    ucs = []
    for song in uss:
        if song in known_songs:
            #print song
            trackmatches = titles[titles["song_id"] == song]
            trackmatches = trackmatches["track_id"].values #could be more than one
            for tm in trackmatches:
                if tm not in btset:
                    track = tracks[tracks["track_id"] == tm]
                    track = track.iloc[:,1:] #leave out track_id
                    cluster = kmeans.predict(track)
                    ucs.append(cluster[0])#only one track used in predict
    userclusters.append(ucs) 
print "predict time:", time.time()-t1

predict time: 46.1766278744


In [64]:
print len(userclusters)

9335


In [65]:
clustercounts = []
clusterfreqs = []
for ucs in userclusters:
    num_clusters = kmeans.cluster_centers_.shape[0]
    indcounts = [0 for cl in range(num_clusters)]
    indfreqs = [0 for cl in range(num_clusters)]
    counts = Counter(ucs)
    countsum = float(sum(counts.values()))
    for name, c in counts.iteritems():
        indcounts[name] = c
        indfreqs[name] = c/countsum
    clustercounts.append(indcounts)
    clusterfreqs.append(indfreqs)
    if len(ucs) > 12:
        print len(ucs), counts
        print indcounts
        print indfreqs

13 Counter({3: 3, 26: 3, 27: 2, 2: 1, 38: 1, 13: 1, 16: 1, 30: 1})
[0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0.07692307692307693, 0.23076923076923078, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.07692307692307693, 0, 0, 0.07692307692307693, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.23076923076923078, 0.15384615384615385, 0, 0, 0.07692307692307693, 0, 0, 0, 0, 0, 0, 0, 0.07692307692307693, 0]
14 Counter({3: 4, 2: 4, 26: 2, 34: 1, 4: 1, 5: 1, 14: 1})
[0, 0, 4, 4, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0.2857142857142857, 0.2857142857142857, 0.07142857142857142, 0.07142857142857142, 0, 0, 0, 0, 0, 0, 0, 0, 0.07142857142857142, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.14285714285714285, 0, 0, 0, 0, 0, 0, 0, 0.07142857142857142, 0, 0, 0, 0, 0]
16 Counter({2: 4, 26: 4, 8: 2, 37: 1, 34: 1, 14: 1, 16: 1, 21: 1, 5: 1})
[0, 0, 4, 0, 0, 1, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 1

In [66]:
with open(basedir+"lastfm_clustercounts.txt", 'w') as cc:
    for clist in clustercounts:
        for c in clist:
            cc.write(str(c) + " ")
        cc.write("\n")

with open(basedir+"lastfm_clusterfreqs.txt", 'w') as cf:
    for flist in clusterfreqs:
        for f in flist:
            cf.write(str(f) + " ")
        cf.write("\n")